### Import dependencies

In [1]:
import os
from dotenv import load_dotenv

from openai import OpenAI

from langchain.agents import AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain_core.tools import tool

from qdrant_client import QdrantClient
from qdrant_client.http.models import Filter, SparseVector
from fastembed import TextEmbedding, SparseTextEmbedding
from langchain.vectorstores import Qdrant

from llama_index.core.evaluation import SemanticSimilarityEvaluator

evaluator = SemanticSimilarityEvaluator()

In [1]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Response
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    CorrectnessEvaluator,
)
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd

pd.set_option("display.max_colwidth", 0)

In [3]:
documents = SimpleDirectoryReader("Data").load_data()

In [ ]:
# create vector index
llm = OpenAI(temperature=0.3, model="gpt-3.5-turbo")
splitter = SentenceSplitter(chunk_size=512)
vector_index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter]
)

### Load environment variables

In [2]:
# Load environment variables
load_dotenv()
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
COLLECTION_NAME = os.getenv("QDRANT_COLLECTION_NAME")

In [3]:
# Initialize dense and sparse embedding models
dense_model = TextEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
sparse_model = SparseTextEmbedding(model_name="Qdrant/bm42-all-minilm-l6-v2-attentions")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

stopwords.txt:   0%|          | 0.00/936 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/91.0M [00:00<?, ?B/s]

In [4]:
# Initialize Qdrant client
qdrant_client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
)


/tmp/ipykernel_5920/2748386960.py:2: UserWarning: Api key is used with an insecure connection.
  qdrant_client = QdrantClient(


In [5]:
# Initialize LLM
llm = ChatOpenAI(temperature=0)


In [6]:
# fetch the prompt from the prompt hub
prompt = hub.pull("hwchase17/openai-functions-agent")

/home/agorricho/anaconda3/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
from qdrant_client import QdrantClient, models
from langchain.agents import tool

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Perform hybrid (dense + sparse) search using Qdrant Query API."""

    # Generate dense and sparse embeddings
    dense_emb = list(dense_model.embed([query]))[0]
    sparse_emb = list(sparse_model.embed([query]))[0]
    sparse_vector = models.SparseVector(
        indices=sparse_emb.indices.tolist(),
        values=sparse_emb.values.tolist()
    )

    
    # Fusion query
    context = qdrant_client.query_points(
        collection_name=COLLECTION_NAME,
        prefetch=[
            models.Prefetch(
                query=sparse_vector,
                using="sparse",
                limit=20,
            ),
            models.Prefetch(
                query=dense_emb,
                using="dense",
                limit=20,
            ),
        ],
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True,
        # with_vector=False,
        limit=5
    )

    # Format result
    serialized = "\n\n".join(
        (f"ID: {pt.id}\
        \nScore: {pt.score}\
        \nText: {pt.payload['text']}\
        \nSource: {pt.payload['file_name']}\
        \nPage: {pt.payload['page_number']}")
        for pt in context.points
    )

    return serialized, context


In [8]:
query='what are the services offered'

In [9]:
serialized = retrieve(query)

/tmp/ipykernel_5920/1996469919.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  serialized = retrieve(query)


In [10]:
print(serialized)

ID: 12        
Score: 0.64285713        
Text: In a Full-Service relationship with a dedicated ﬁnancial professional, you have access to a full suite of TIS brokerage and TAS 
investment advisory products and services. Minimum asset balance requirements apply. You have a direct relationship with
your ﬁnancial professional for investment strategies, recommendations, and guidance speciﬁc to your needs and goals. Full- 
Service brokerage accounts are eligible to invest in equities, ﬁxed income, ETFs, mutual funds, annuities, structured products,
alternative investments, and other securities on our Platform which TIS makes available to such accounts. 
In a Full-Service relationship with the Client Advisory Center (“CAC”), you have access to a more limited menu of TIS brokerage
products and services from which a CAC ﬁnancial professional can make recommendations, including mutual funds, annuities,
brokered CDs, Treasuries, structured products, and certain advisory programs oﬀered through TA

In [11]:
dense_emb = list(dense_model.embed([query]))[0]
len(dense_emb)

384

In [12]:
dense_emb[:10]

array([-0.05865774, -0.02567403,  0.02276097, -0.03343063, -0.05385467,
        0.06762547,  0.00263454, -0.01720169,  0.0195535 , -0.01091614])

In [13]:
sparse_embedding = list(sparse_model.embed([query]))[0]

In [14]:
sparse_embedding

SparseEmbedding(values=array([0.34146956, 0.26927151]), indices=array([1342301861, 1401524087]))

In [15]:
# Create sparse vector
sparse_vector = SparseVector(
    indices=sparse_embedding.indices.tolist(),
    values=sparse_embedding.values.tolist()
    )

In [16]:
sparse_vector

SparseVector(indices=[1342301861, 1401524087], values=[0.34146955540420515, 0.2692715119050256])

In [17]:
# Fusion query
response = qdrant_client.query_points(
    collection_name=COLLECTION_NAME,
    prefetch=[
        models.Prefetch(
            query=sparse_vector,
            using="sparse",
            limit=20,
        ),
        models.Prefetch(
            query=dense_emb,
            using="dense",
            limit=20,
        ),
    ],
    query=models.FusionQuery(fusion=models.Fusion.RRF),
    with_payload=True,
    # with_vector=False,
    limit=5
)

In [18]:
response

QueryResponse(points=[ScoredPoint(id=12, version=0, score=0.64285713, payload={'text': 'In a Full-Service relationship with a dedicated ﬁnancial professional, you have access to a full suite of TIS brokerage and TAS \ninvestment advisory products and services. Minimum asset balance requirements apply. You have a direct relationship with\nyour ﬁnancial professional for investment strategies, recommendations, and guidance speciﬁc to your needs and goals. Full- \nService brokerage accounts are eligible to invest in equities, ﬁxed income, ETFs, mutual funds, annuities, structured products,\nalternative investments, and other securities on our Platform which TIS makes available to such accounts. \nIn a Full-Service relationship with the Client Advisory Center (“CAC”), you have access to a more limited menu of TIS brokerage\nproducts and services from which a CAC ﬁnancial professional can make recommendations, including mutual funds, annuities,\nbrokered CDs, Treasuries, structured products,

In [19]:
len(response.points)

5

In [20]:
response.points[4].score

0.33333334

In [21]:
# Format result
serialized_1 = "\n\n".join(
    (f"ID: {pt.id}\
     \nScore: {pt.score}\
     \nText: {pt.payload['text']}\
     \nSource: {pt.payload['file_name']\
     }")
    for pt in response.points
)

print(serialized_1)

ID: 12     
Score: 0.64285713     
Text: In a Full-Service relationship with a dedicated ﬁnancial professional, you have access to a full suite of TIS brokerage and TAS 
investment advisory products and services. Minimum asset balance requirements apply. You have a direct relationship with
your ﬁnancial professional for investment strategies, recommendations, and guidance speciﬁc to your needs and goals. Full- 
Service brokerage accounts are eligible to invest in equities, ﬁxed income, ETFs, mutual funds, annuities, structured products,
alternative investments, and other securities on our Platform which TIS makes available to such accounts. 
In a Full-Service relationship with the Client Advisory Center (“CAC”), you have access to a more limited menu of TIS brokerage
products and services from which a CAC ﬁnancial professional can make recommendations, including mutual funds, annuities,
brokered CDs, Treasuries, structured products, and certain advisory programs oﬀered through TAS. Ind

In [22]:
serialized_2 = "\n\n".join(
    (f"Source: {pt.payload['file_name']}\
     \n" f"Content: {pt.payload['text']}")
    for pt in response.points
    )

print(serialized_2)

Source: tis-investing-guide.pdf     
Content: In a Full-Service relationship with a dedicated ﬁnancial professional, you have access to a full suite of TIS brokerage and TAS 
investment advisory products and services. Minimum asset balance requirements apply. You have a direct relationship with
your ﬁnancial professional for investment strategies, recommendations, and guidance speciﬁc to your needs and goals. Full- 
Service brokerage accounts are eligible to invest in equities, ﬁxed income, ETFs, mutual funds, annuities, structured products,
alternative investments, and other securities on our Platform which TIS makes available to such accounts. 
In a Full-Service relationship with the Client Advisory Center (“CAC”), you have access to a more limited menu of TIS brokerage
products and services from which a CAC ﬁnancial professional can make recommendations, including mutual funds, annuities,
brokered CDs, Treasuries, structured products, and certain advisory programs oﬀered through TAS

In [23]:
# combine the tools and provide to the llm
tools = [retrieve]
agent = create_tool_calling_agent(llm, tools, prompt)

# create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [24]:
# invoke the agent
response = agent_executor.invoke({"input": "which are the services offered by the company?"})



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************0ThB. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
print(response['output'])

The company offers a range of services including brokerage services, investment advisory products, annuity products, securities-backed lending, and more. If you need more specific details about any particular service, feel free to ask!


In [ ]:
# invoke the agent
response = agent_executor.invoke({"input": "which are the main risk factors highlighted in the document?"})



> Entering new AgentExecutor chain...

Invoking: `retrieve` with `{'query': 'main risk factors'}`


ID: 79        
Score: 0.6666667        
Text: Prior to investing in a structured product, please read the issuer’s offering documents, prospectus, and supplements, including 
the “Risk Factors” described therein, to determine and ensure that the product is suitable for your investment needs and that
you have the ability to bear the economic and other risks inherent to the product. 
Conﬂicts of Interest – Structured Products 
Compensation to financial professionals is generated from sales concessions. Sales concessions range from 1% to 3.5%,
depending on the term length of the structured product. In addition, an underlying index or set of investments (called the
underlier) which are used to calculate the performance of a structured product can have a management fee. Such 
management fees range from 0% to 1% depending on the nature of each underlier.        
Source: tis-investing-guide.p

In [ ]:
print(response['output'])

The main risk factors highlighted in the document include:

1. Credit Risk
2. Liquidity Risk
3. Price Risk
4. Call Risk
5. Coupon Risk
6. Tax Considerations

These risks and considerations are associated with the purchase of structured products. It is important to refer to the offering documents associated with each structured product issuance for a detailed explanation of these risks and considerations.


### Appendix

Qdrant documentation at https://api.qdrant.tech/api-reference/search/query-points

In [ ]:
from qdrant_client import QdrantClient, models

client = QdrantClient(url="http://localhost:6333")

# Query nearest by ID
nearest = client.query_points(
    collection_name="{collection_name}",
    query="43cf51e2-8777-4f52-bc74-c2cbde0c8b04",
)

# Recommend on the average of these vectors
recommended = client.query_points(
    collection_name="{collection_name}",
    query=models.RecommendQuery(recommend=models.RecommendInput(
        positive=["43cf51e2-8777-4f52-bc74-c2cbde0c8b04", [0.11, 0.35, 0.6, ...]],
        negative=[[0.01, 0.45, 0.67, ...]]
    ))
)

# Fusion query
hybrid = client.query_points(
    collection_name="{collection_name}",
    prefetch=[
        models.Prefetch(
            query=models.SparseVector(indices=[1, 42], values=[0.22, 0.8]),
            using="sparse",
            limit=20,
        ),
        models.Prefetch(
            query=[0.01, 0.45, 0.67, ...],  # <-- dense vector
            using="dense",
            limit=20,
        ),
    ],
    query=models.FusionQuery(fusion=models.Fusion.RRF),
)

# 2-stage query
refined = client.query_points(
    collection_name="{collection_name}",
    prefetch=models.Prefetch(
        query=[0.01, 0.45, 0.67, ...],  # <-- dense vector
        limit=100,
    ),
    query=[
        [0.1, 0.2, ...],  # <─┐
        [0.2, 0.1, ...],  # < ├─ multi-vector
        [0.8, 0.9, ...],  # < ┘
    ],
    using="colbert",
    limit=10,
)

# Random sampling (as of 1.11.0)
sampled = client.query_points(
    collection_name="{collection_name}",
    query=models.SampleQuery(sample=models.Sample.RANDOM)
)

# Score boost depending on payload conditions (as of 1.14.0)
tag_boosted = client.query_points(
    collection_name="{collection_name}",
    prefetch=models.Prefetch(
        query=[0.2, 0.8, ...],  # <-- dense vector
        limit=50
    ),
    query=models.FormulaQuery(
        formula=models.SumExpression(sum=[
            "$score",
            models.MultExpression(mult=[0.5, models.FieldCondition(key="tag", match=models.MatchAny(any=["h1", "h2", "h3", "h4"]))]),
            models.MultExpression(mult=[0.25, models.FieldCondition(key="tag", match=models.MatchAny(any=["p", "li"]))])
        ]
    ))
)

# Score boost geographically closer points (as of 1.14.0)
geo_boosted = client.query_points(
    collection_name="{collection_name}",
    prefetch=models.Prefetch(
        query=[0.2, 0.8, ...],  # <-- dense vector
        limit=50
    ),
    query=models.FormulaQuery(
        formula=models.SumExpression(sum=[
            "$score",
            models.GaussDecayExpression(
                gauss_decay=models.DecayParamsExpression(
                    x=models.GeoDistance(
                        geo_distance=models.GeoDistanceParams(
                            origin=models.GeoPoint(
                                lat=52.504043,
                                lon=13.393236
                            ),  # Berlin
                            to="geo.location"
                        )
                    ),
                    scale=5000  # 5km
                )
            )
        ]),
        defaults={"geo.location": models.GeoPoint(lat=48.137154, lon=11.576124)}  # Munich
    )
)
